In [1]:
import torch
import torch.nn as nn

In [107]:
class Conv2D(nn.Module):
    def __init__(self, kernel_size, input_channel, output_channel, stride=0, padding=0):
        super().__init__()
        self.kernel = nn.Parameter(torch.rand((output_channel, kernel_size[0], kernel_size[1])))
        self.bias = nn.Parameter(torch.rand(output_channel))
        self.input_channel = input_channel
        self.output_channel = output_channel
        self.kernel_size = kernel_size
    
    def conv(self, X, kernel):
        height, width = self.kernel_size
        Y = torch.zeros((X.shape[0] - height + 1, X.shape[1] - width + 1))
        for i in range(Y.shape[0]):
            for j in range(Y.shape[1]):
                cube = X[i : i + height, j : j + width]
                Y[i][j] = (cube * kernel).sum()
        return Y
        
    def forward(self, X):
        assert(X.shape[0] == self.input_channel)
        Ys = torch.zeros((self.output_channel, X.shape[1] - self.kernel_size[0] + 1, X.shape[2] - self.kernel_size[1] + 1), dtype=torch.float)
        for i in range(self.output_channel):
            Y = Ys[i]
            for j in range(self.input_channel):
                Y += self.conv(X[j], self.kernel[i])
            Y += self.bias[i]
        return Ys

In [218]:
# X is the map, 1 means higher part, 0 represent lower part
# Y is the ground truth, the second column = 1, means the 1st and the 2nd column of X is a boarder
X = torch.tensor([[[1, 1, 0, 0, 0], [1, 1, 0, 0, 0]]])
Y = torch.tensor([[[0, 1, 0, 0, ], [0, 1, 0, 0]]])

# training a vertical boarder detector
# conv is a convenlutional layer which will be trained to detect the boarder
conv = Conv2D((1, 2), 1, 1)
opt = torch.optim.SGD(conv.parameters(), lr=0.01)


In [223]:
# training
for i in range(5000):
    conv.zero_grad()
    Y_hat = conv(X)
    loss = (Y - Y_hat) ** 2
    loss.sum().backward()
    opt.step()

print(conv)
print(conv(X))
print(conv.kernel, conv.bias)
print(conv.input_channel, conv.output_channel, conv.kernel_size)

Conv2D()
tensor([[[6.9539e-07, 1.0000e+00, 3.3776e-07, 3.3776e-07],
         [6.9539e-07, 1.0000e+00, 3.3776e-07, 3.3776e-07]]],
       grad_fn=<CopySlices>)
Parameter containing:
tensor([[[ 1.0000, -1.0000]]], requires_grad=True) Parameter containing:
tensor([3.3776e-07], requires_grad=True)
1 1 (1, 2)
